# Tourism Data Clean-Up and Integration

## Introduction: 

The dataset provided for this case study comprises international tourist arrival records for August 2023. In this project, we will embark on a journey of data cleaning and integration. The primary objective is to standardize the given dataset, ensuring consistency and accuracy, to facilitate its seamless integration into a historical record file named "Tourists, Purpose (2)." 


## Loading the data

In [118]:
import pandas as pd

#  We start bt loading the Excel file and exploring the tabs
file_path ='Tables, August 2023.xlsx'
xl = pd.ExcelFile(file_path)
print(xl.sheet_names) 



['MASTERLIST', 'Tourists, Purpose (2)', 'Tourists, Purpose', 'Tourists, Purpose_Merged']


#### We will now take a look at the individual tabs to see the nature of the data by looking at the first 5 rows of the tables

### MASTERLIST

In [16]:
df_sheet1 = pd.read_excel('Tables, August 2023.xlsx', sheet_name=0)

In [17]:
print(df_sheet1.head())

     Country Country / Source  Country.1 Country / Source.1 ContinentStatsSA  \
0    Algeria          Algeria    Algeria            Algeria           AFRICA   
1     Angola           Angola     Angola             Angola           AFRICA   
2  Ascension        Ascension  Ascension          Ascension           AFRICA   
3      Benin            Benin      Benin              Benin           AFRICA   
4   Botswana         Botswana   Botswana           Botswana           AFRICA   

  RegionStatsSA DESIGNATION        SOURCE     CATEGORY  
0    AFRICA AIR      AFRICA  OTHER AFRICA   AFRICA AIR  
1    AFRICA AIR      AFRICA        ANGOLA   AFRICA AIR  
2    AFRICA AIR      AFRICA  OTHER AFRICA   AFRICA AIR  
3    AFRICA AIR      AFRICA  OTHER AFRICA   AFRICA AIR  
4   AFRICA LAND      AFRICA      BOTSWANA  AFRICA LAND  


### Tourists, Purpose (2)

In [9]:
df_sheet2 = pd.read_excel('Tables, August 2023.xlsx', sheet_name=1)

In [10]:
print(df_sheet2.head())

   Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3      Row Labels   Unnamed: 5  \
0        2023    Purpose        Jul 2023-07-01           Total  GRAND TOTAL   
1        2023    Purpose        Jul 2023-07-01  Overseas Total       REGION   
2        2023    Purpose        Jul 2023-07-01          Europe       REGION   
3        2023    Purpose        Jul 2023-07-01         Albania       EUROPE   
4        2023    Purpose        Jul 2023-07-01         Andorra       EUROPE   

    Unnamed: 6      Unnamed: 7   Unnamed: 8    Unnamed: 9  Unnamed: 10  \
0  GRAND TOTAL           Total  GRAND TOTAL   GRAND TOTAL  GRAND TOTAL   
1       REGION  Overseas Total       REGION        REGION       REGION   
2       REGION          Europe       REGION        REGION       REGION   
3       EUROPE         Albania       EUROPE  OTHER EUROPE       EUROPE   
4       EUROPE         Andorra       EUROPE  OTHER EUROPE       EUROPE   

   Grand Total  Grand Total.1  Business  Holiday   Study  Medical treatment  
0 

###  Tourists, Purpose

In [48]:
df_sheet3 = pd.read_excel('Tables, August 2023.xlsx', sheet_name=2, skiprows=5)

In [49]:
print(df_sheet3.head())

  Row Labels  01 Business  02 Holiday  03 Study  09 Medical treatment  \
0  1Overseas       2489.0      162341     814.0                  61.0   
1    1Europe        974.0       92174     424.0                  30.0   
2    Albania          NaN           7       NaN                   NaN   
3    Andorra          NaN          24       NaN                   NaN   
4    Austria         13.0        1239       6.0                   NaN   

   Grand Total  
0       165705  
1        93602  
2            7  
3           24  
4         1258  


In [46]:
pivot_df = pd.read_excel('Tables, August 2023.xlsx', sheet_name=2, skiprows=5)  
print(pivot_df.head())

  Row Labels  01 Business  02 Holiday  03 Study  09 Medical treatment  \
0  1Overseas       2489.0      162341     814.0                  61.0   
1    1Europe        974.0       92174     424.0                  30.0   
2    Albania          NaN           7       NaN                   NaN   
3    Andorra          NaN          24       NaN                   NaN   
4    Austria         13.0        1239       6.0                   NaN   

   Grand Total  
0       165705  
1        93602  
2            7  
3           24  
4         1258  


## Standardizing country and region names

In [96]:

pivot_df = pd.read_excel('Tables, August 2023.xlsx', sheet_name=2, skiprows=5)  

countries_list = pivot_df.iloc[6:146, 0].tolist()

country_mapping = {
    '1Overseas': 'Overseas',
    '1Europe': 'Europe',
    '8ASADC': 'SADC Countries'  
}

def clean_country_name(name):
    
    regional_groupings = {
        '8ASADC': 'SADC Countries',
       
    }
    
   
    if name in regional_groupings:
        return regional_groupings[name]
    # Remove numerical prefixes and special characters
    cleaned_name = name.strip('0123456789.-,;:')
    
    # Correct typographical errors
    cleaned_name = cleaned_name.replace('Usa', 'USA')
    
    # Clarify regional groupings
    if cleaned_name in country_mapping:
        return country_mapping[cleaned_name]
    
    return cleaned_name

# Apply the clean_country_name function to standardize country names
pivot_df['Row Labels'] = pivot_df['Row Labels'].apply(clean_country_name)


pivot_df.to_excel('standardized_pivot_data.xlsx', index=False)


print(pivot_df.head(147))

         Row Labels  01 Business  02 Holiday  03 Study  09 Medical treatment  \
0          Overseas       2489.0      162341     814.0                  61.0   
1            Europe        974.0       92174     424.0                  30.0   
2           Albania          NaN           7       NaN                   NaN   
3           Andorra          NaN          24       NaN                   NaN   
4           Austria         13.0        1239       6.0                   NaN   
..              ...          ...         ...       ...                   ...   
142  SADC Countries      17833.0      510313    2308.0                 171.0   
143          Angola         26.0        3491      71.0                  28.0   
144        Botswana       1070.0       24854     225.0                  18.0   
145             DRC         72.0        2014     100.0                  27.0   
146        Eswatini       2567.0       64196     182.0                   6.0   

     Grand Total  
0         165705  
1

## Clean Purpose Columns

In [97]:
# 2. Clean purpose columns

pivot_df.columns = [col.strip().replace('Purpose', '').replace(' ', '') for col in pivot_df.columns]
purpose_columns = [col for col in pivot_df.columns if col.startswith('Purpose')]
pivot_df[purpose_columns] = pivot_df[purpose_columns].apply(lambda x: x.str.replace(r'\d+', '').str.strip())


print(pivot_df.head())



  RowLabels  01Business  02Holiday  03Study  09Medicaltreatment  GrandTotal
0  Overseas      2489.0     162341    814.0                61.0      165705
1    Europe       974.0      92174    424.0                30.0       93602
2   Albania         NaN          7      NaN                 NaN           7
3   Andorra         NaN         24      NaN                 NaN          24
4   Austria        13.0       1239      6.0                 NaN        1258


In [98]:

# Remove numerical prefixes from purpose column names
prefix_length = 2 
pivot_df.columns = [col[prefix_length:].strip() if col[:prefix_length].isdigit() else col for col in pivot_df.columns]


pivot_df.to_excel('standardized_pivot_data.xlsx', index=False)


print(pivot_df.head())


  RowLabels  Business  Holiday  Study  Medicaltreatment  GrandTotal
0  Overseas    2489.0   162341  814.0              61.0      165705
1    Europe     974.0    92174  424.0              30.0       93602
2   Albania       NaN        7    NaN               NaN           7
3   Andorra       NaN       24    NaN               NaN          24
4   Austria      13.0     1239    6.0               NaN        1258


# Data Integration 

## Create new columns for Year, Month, and Date

In [102]:
# Create new columns for Year, Month, and Date
pivot_df['Year'] = 2023
pivot_df['Month'] = 'Aug'
pivot_df['Date'] = '2023-08'

pivot_df.to_excel('standardized_pivot_data.xlsx', index=False)

print(pivot_df.head())  


  RowLabels  Business  Holiday  Study  Medicaltreatment  GrandTotal  Year  \
0  Overseas    2489.0   162341  814.0              61.0      165705  2023   
1    Europe     974.0    92174  424.0              30.0       93602  2023   
2   Albania       NaN        7    NaN               NaN           7  2023   
3   Andorra       NaN       24    NaN               NaN          24  2023   
4   Austria      13.0     1239    6.0               NaN        1258  2023   

  Month     Date  
0   Aug  2023-08  
1   Aug  2023-08  
2   Aug  2023-08  
3   Aug  2023-08  
4   Aug  2023-08  


## Intergrating 'Tourist, Purpose' into the 'Tourist, Purpose (2)'

In [113]:

# Load the cleaned current dataset 
df_current = pd.read_excel('standardized_pivot_data.xlsx')

# Load the historical dataset 
df_historical = pd.read_excel('Tables, August 2023.xlsx', sheet_name=1)


# Clean data in df_historical
df_historical = df_historical.drop_duplicates()  # Remove duplicate rows
# Remove spaces from column names
df_historical.columns = df_historical.columns.str.replace(' ', '')



print("Columns in df_current:", df_current.columns.tolist())
print("Columns in df_historical:", df_historical.columns.tolist())


Columns in df_current: ['RowLabels', 'Business', 'Holiday', 'Study', 'Medicaltreatment', 'GrandTotal', 'Year', 'Month', 'Date']
Columns in df_historical: ['Unnamed:0', 'Unnamed:1', 'Unnamed:2', 'Unnamed:3', 'RowLabels', 'Unnamed:5', 'Unnamed:6', 'Unnamed:7', 'Unnamed:8', 'Unnamed:9', 'Unnamed:10', 'GrandTotal', 'GrandTotal.1', 'Business', 'Holiday', 'Study', 'Medicaltreatment']


 Append the cleaned data to the historical data

In [116]:
# Merge the current dataset with the  historical dataset

merged_df = pd.merge(df_current, df_historical, on='RowLabels', how='outer')


merged_df.to_excel('Tourists, Purpose_Merged.xlsx', index=False)


print(merged_df.head())


  RowLabels  Business_x  Holiday_x  Study_x  Medicaltreatment_x  GrandTotal_x  \
0  Overseas      2489.0   162341.0    814.0                61.0      165705.0   
1    Europe       974.0    92174.0    424.0                30.0       93602.0   
2   Albania         NaN        7.0      NaN                 NaN           7.0   
3   Andorra         NaN       24.0      NaN                 NaN          24.0   
4   Austria        13.0     1239.0      6.0                 NaN        1258.0   

     Year Month     Date  Unnamed:0  ... Unnamed:7 Unnamed:8     Unnamed:9  \
0  2023.0   Aug  2023-08        NaN  ...       NaN       NaN           NaN   
1  2023.0   Aug  2023-08     2023.0  ...    Europe    REGION        REGION   
2  2023.0   Aug  2023-08     2023.0  ...   Albania    EUROPE  OTHER EUROPE   
3  2023.0   Aug  2023-08     2023.0  ...   Andorra    EUROPE  OTHER EUROPE   
4  2023.0   Aug  2023-08     2023.0  ...   Austria    EUROPE       AUSTRIA   

  Unnamed:10 GrandTotal_y GrandTotal.1 Busin

## Load the appended sheet into the workbook

In [117]:
from openpyxl import load_workbook

# File paths
original_file_path = 'Tables, August 2023.xlsx'
merged_file_path = 'Tourists, Purpose_Merged.xlsx'

# Load the original Excel file and merged data file
original_wb = load_workbook(filename=original_file_path)
merged_wb = load_workbook(filename=merged_file_path)

# Get the active sheet from the merged data
merged_sheet = merged_wb.active

# Create a new sheet in the original workbook
new_sheet_name = 'Tourists, Purpose_Merged'
new_sheet = original_wb.create_sheet(title=new_sheet_name)

# Copy data from the merged sheet to the new sheet in the original workbook
for row in merged_sheet.iter_rows(values_only=True):
    new_sheet.append(row)


original_wb.save(filename=original_file_path)

print(f"Merged data loaded into '{original_file_path}' in sheet '{new_sheet_name}'.")


Merged data loaded into 'Tables, August 2023.xlsx' in sheet 'Tourists, Purpose_Merged'.
